In [1]:
#%pip install -r requirements.txt

In [109]:
import pandas as pd
import json
import pickle
import helpers
from tqdm.notebook import tqdm
import numpy as np
from nltk.stem import PorterStemmer
import multiprocessing as mp
import os 
import math
from functools import partial

DATA_DIR = 'data'


stemmer = PorterStemmer()
# Load the data from files
with open(f'{DATA_DIR}/corpus.jsonl', 'r') as f:
    corpus_data = {int(item['_id']): item['text'] for item in (json.loads(line) for line in f)}

with open(f'{DATA_DIR}/queries.jsonl', 'r') as f:
    queries_data = {int(item['_id']): item['text'] for item in (json.loads(line) for line in f)}

train_data = pd.read_csv(f'{DATA_DIR}/task1_train.tsv', delimiter='\t')
test_data = pd.read_csv(f'{DATA_DIR}/task1_test.tsv', delimiter='\t')

# Rename corpus-id to document_id and query-id to query_id in both train and test data
train_data = train_data.rename(columns={'corpus-id': 'document_id', 'query-id': 'query_id'})
test_data = test_data.rename(columns={'corpus-id': 'document_id', 'query-id': 'query_id'})
# Make sure that the document_id and query_id are int64
train_data['document_id'] = train_data['document_id'].astype('int64')
train_data['query_id'] = train_data['query_id'].astype('int64')

# Create a df from the corpus data
corpus_df = pd.DataFrame.from_dict(corpus_data, orient='index', columns=['text'])
# Create a df from the queries data
queries_df = pd.DataFrame.from_dict(queries_data, orient='index', columns=['text'])

# Check if documents.pkl exists:
if os.path.isfile(f'{DATA_DIR}/documents.pkl'):
    print('Loading tokenized documents from pickle file...')
    # load the tokenized documents from pickle file
    with open(f'{DATA_DIR}/documents.pkl', 'rb') as f:
        documents = pickle.load(f)
else:
    print('File not found. Tokenizing documents...')
    documents = corpus_df['text'].tolist()
    documents = [x.strip() for x in documents]
    # use multiprocessing to speed up the process
    pool = mp.Pool(mp.cpu_count())
    # pass both documents and stemmer as arguments to the tokenize function
    fn = partial(helpers.tokenize, stemmer=stemmer)   
    documents = list(tqdm(pool.imap(fn, documents), total=len(documents))) 
    # save the tokenized documents as pickle file
    with open(f'{DATA_DIR}/documents.pkl', 'wb') as f:
        pickle.dump(documents, f)

Loading tokenized documents from pickle file...


In [110]:
vocabulary = list(set([item for sublist in documents for item in sublist]))
vocabulary.sort()

In [111]:
# Compute the number of documents that contain each word
doc_freqs = {}
for doc in tqdm(documents):
    for word in set(doc):
        doc_freqs[word] = doc_freqs.get(word, 0) + 1

  0%|          | 0/1471406 [00:00<?, ?it/s]

In [112]:
# Compute the IDF for each word in the vocabulary
num_docs = len(documents)
#idfs = {word: math.log(num_docs / freq) for word, freq in doc_freqs.items()}
idfs = {word: math.log((num_docs - freq + 0.5)/(freq + 0.5)+1) for word, freq in doc_freqs.items()}

In [113]:
# create a vocabulary dictionary with the index of each word in the vocabulary
vocabulary_dict = {word: i for i, word in tqdm(enumerate(vocabulary))}

0it [00:01, ?it/s]

In [114]:
# find average document length
avg_doc_length = sum([len(doc) for doc in documents]) / len(documents)

In [115]:
# now get the first row of the query matrix
query = queries_df['text'].iloc[0]

In [116]:
# tokenize the query
query = helpers.tokenize(query, stemmer)

In [117]:
scores = []
for doc in tqdm(documents, desc="Scoring documents..."):
    score = helpers.bm25(doc, query, idfs, avg_doc_length, k1=1, b=0.75)
    scores.append(score)

Scoring documents...:   0%|          | 0/1471406 [00:00<?, ?it/s]

In [118]:
# get the documents with the highest scores (top 10)
top_10 = np.argsort(scores)[::-1][:10]

In [119]:
for index in top_10:
    print(f'Document ID: {corpus_df.index.values[index]}')
    print(f'Text: "{corpus_df.iloc[index].text}"')
    print(f'Score: {scores[index]}')
    print()

Document ID: 7306165
Text: "He is honest-but smart as hell.. Meanwhile, President Truman was told of the successful test of the Manhattan Project (atomic bomb) in Alamogordo, New Mexico on Jul 16, 1945. Diary of President Truman of Jul 18, 1945 shows Discussed Manhattan (it is a success).e is honest-but smart as hell.. Meanwhile, President Truman was told of the successful test of the Manhattan Project (atomic bomb) in Alamogordo, New Mexico on Jul 16, 1945. Diary of President Truman of Jul 18, 1945 shows Discussed Manhattan (it is a success)."
Score: 22.80581471839291

Document ID: 0
Text: "The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated."
Score: 19.472276735865158

Document ID: 2036644
Text: "Manhattan Project. 

In [120]:
# Since finding all the scores for each query takes a long time,
# we will use matrix multiplication to find the scores for all the queries at once

k1 = 1
b = 0.75

# first create the document matrix where each row is a document
# and each column is a word
# the value of the cell is idfs[term] * (counts[term] * (k1 + 1) / (counts[term] + k1 * (1 - b + b * len(doc) / avg_doc_length)))
from scipy.sparse import lil_matrix

# check if doc_matrix.pkl exists
if os.path.isfile(f'{DATA_DIR}/doc_matrix.pkl'):
    print('Loading document matrix from pickle file...')
    # load the document matrix from pickle file
    with open(f'{DATA_DIR}/doc_matrix.pkl', 'rb') as f:
        doc_matrix = pickle.load(f)
else:
    # Compute the doc matrix
    doc_matrix = lil_matrix((len(documents), len(vocabulary))) # We use lil_matrix since it is efficient in incremental assignments
    for doc_id, doc in tqdm(enumerate(documents), desc="Computing document matrix"):
        counts, max_count = helpers.count_terms(doc)
        for term, count in counts.items():
            if term in vocabulary_dict:
                term_id = vocabulary_dict[term]
                doc_matrix[doc_id, term_id] = idfs[term] * (counts[term] * (k1 + 1) / (counts[term] + k1 * (1 - b + b * len(doc) / avg_doc_length)))

    # Save the doc matrix as a pickle file
    with open(f'{DATA_DIR}/doc_matrix.pkl', 'wb') as f:
        pickle.dump(doc_matrix, f)

Loading document matrix from pickle file...


In [121]:
doc_matrix

<1471406x1130369 sparse matrix of type '<class 'numpy.float64'>'
	with 40408661 stored elements in List of Lists format>

In [122]:
doc_matrix = doc_matrix.tocsr()

In [123]:
test_data

,id,query_id
0,0,300674
1,1,125705
2,2,94798
3,3,9083
4,4,174249
...,...,...
7432,7432,147073
7433,7433,243761
7434,7434,162662
7435,7435,247194


In [124]:
# Now create the query matrix where each row is a query
# and each column is a word
# the value of the cell is 1 if the word is in the query, 0 otherwise
query_matrix = lil_matrix((len(test_data), len(vocabulary)))
# iterate row by row
for index, row in tqdm(test_data.iterrows(), desc="Computing query matrix"):
    # get the query from query_id
    query_id = row['query_id']
    query = queries_df.loc[query_id]
    query = helpers.tokenize(query['text'], stemmer)
    for term in query:
        if term in vocabulary_dict:
            term_id = vocabulary_dict[term]
            query_matrix[index, term_id] = 1

query_matrix = query_matrix.tocsr()

Computing query matrix: 0it [00:00, ?it/s]

In [125]:
# Now we can compute the scores for all the queries at once
scores = query_matrix.dot(doc_matrix.T)
# This gives us a matrix of shape (num_queries, num_documents)
# Each cell contains the score of a document for a query

In [126]:
from scipy.sparse import csr_matrix

# Find the indices of the top 3 elements along each row
k = 10
top_k_indices = np.zeros((scores.shape[0], k), dtype=int)
for i in range(scores.shape[0]):
    row = scores.getrow(i).toarray()[0]
    top_k_indices[i] = np.argpartition(row, -k)[-k:]

In [127]:
# Get the document ids of the top k documents for each query
top_k_doc_ids = []
top_k_scores = []
for index, row in enumerate(top_k_indices):
    top_k_doc_ids.append([corpus_df.index.values[i] for i in row])
    top_k_scores.append([scores[index, i] for i in row])

In [128]:
top_k_scores[0]

[30.41874710329342,
 31.99697828271954,
 31.726511480146954,
 36.30561337354201,
 50.13671280047856,
 52.375477845441964,
 36.905421177995265,
 43.60453799353136,
 50.11667425529357,
 46.69823514899414]

In [129]:
for index, row in test_data.iterrows():
    print(f'QueryID: {row["query_id"]}:')
    print(f'Query: "{queries_df.loc[row["query_id"]]["text"]}"')
    print(f'Top {k} documents:')
    for doc_id in top_k_doc_ids[index]:
        print(f'\t Document ID: {doc_id}')
        print(f'\t Text: "{corpus_df.loc[doc_id]["text"]}"')
        # find row index of doc_id
        row_index = corpus_df.index.get_loc(doc_id)
        print(f'\t Score: {scores[index, row_index]}')
        print()
    break

QueryID: 300674:
Query: "how many years did william bradford serve as governor of plymouth colony?"
Top 10 documents:
	 Document ID: 4501973
	 Text: "(The Mayflower in Plymouth harbour by William Halsall, 1882). On this day in 1620, William Bradford and the Mayflower Pilgirms landed at Plymouth Rock in Plymouth, Massachusetts. The Mayflower transported the first English Pilgrims to America, with 102 passengers."
	 Score: 30.41874710329342

	 Document ID: 3872942
	 Text: "Plymouth Colony was founded by a group of people English separatists who later came to be known as the Pilgrims. The core group (roughly 40% of the adults and 56% of the family groupings) was part of a Congregationalist congregation led by William Bradford."
	 Score: 31.99697828271954

	 Document ID: 3305011
	 Text: "In addition, they settled as families for the most part, unique in Atlantic coast settlement at this point. Here we read from the journal of the colony's longtime governor, William Bradford, of the colonis

In [130]:
train_data_2 = pd.read_csv(f'{DATA_DIR}/task2_train.tsv', delimiter='\t')
test_data_2 = pd.read_csv(f'{DATA_DIR}/task2_test.tsv', delimiter='\t')

In [131]:
#Since the corpus-id and score columns contains string values,
#this function converts them to a list of integers
def extract_numbers(ids_str):
    splitted = ids_str.split(", ")
    #Remove the square brackets in the first and last element
    splitted[0] = splitted[0][1:]
    splitted[-1] = splitted[-1][:-1]
    return [int(id) for id in splitted]

In [132]:
train_data_2

,query-id,corpus-id,score
0,915593,"[1396701, 1396704, 1396705, 1396707, 1396708, ...","[0, 0, 1, 0, 2, 0, 3, 0, 0, 0, 2, 1, 2, 0, 0, ..."
1,146187,"[1028971, 1028972, 1131101, 1138801, 1230566, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1114646,"[1002453, 1216492, 1316103, 1316109, 1342262, ...","[0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, ..."
3,1129237,"[1020793, 1128332, 1138726, 1169301, 120308, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 3, 0, ..."
4,573724,"[1005338, 104856, 1053303, 1165128, 1165129, 1...","[1, 1, 0, 0, 1, 0, 0, 2, 2, 0, 0, 0, 1, 0, 0, ..."
5,148538,"[1299824, 1299830, 1311202, 1311204, 1311206, ...","[2, 1, 2, 1, 0, 1, 1, 2, 1, 2, 2, 2, 0, 1, 1, ..."
6,527433,"[1000485, 1101462, 1187918, 1212778, 1212782, ...","[3, 0, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 3, ..."
7,130510,"[1046258, 1110766, 1156210, 1159414, 1211365, ...","[0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 2, 2, ..."
8,405717,"[1111371, 1111372, 1111375, 1538943, 1538949, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
9,1106007,"[1020463, 1040867, 1195441, 1334328, 1334330, ...","[1, 0, 0, 2, 3, 2, 0, 2, 3, 3, 1, 0, 0, 0, 0, ..."


In [167]:
# Now create the query matrix where each row is a query
# and each column is a word
# the value of the cell is 1 if the word is in the query, 0 otherwise
query_matrix_2 = lil_matrix((len(test_data_2), len(vocabulary)))
# iterate row by row
for index, row in tqdm(test_data_2.iterrows(), desc="Computing query matrixvfor task 2"):
    # get the query from query_id
    query_id = row['query-id']
    query = queries_df.loc[query_id]
    query = helpers.tokenize(query['text'], stemmer)
    for term in query:
        if term in vocabulary_dict:
            term_id = vocabulary_dict[term]
            query_matrix_2[index, term_id] = 1

query_matrix_2 = query_matrix_2.tocsr()

Computing query matrixvfor task 2: 0it [00:00, ?it/s]

In [168]:
scores_task_2 = query_matrix_2.dot(doc_matrix.T)

In [169]:
scores_task_2[0]

<1x1471406 sparse matrix of type '<class 'numpy.float64'>'
	with 2377 stored elements in Compressed Sparse Row format>

In [179]:
for index, row in test_data_2.iterrows():
    print(f'QueryID: {row["query-id"]}:')
    print(f'Query: "{queries_df.loc[row["query-id"]]["text"]}"')
    print(f'Top {k} documents:')

    corpus_ids = extract_numbers(row["corpus-id"])
    print(row_index)
    sorted_scores_task_2 = []
    for doc_id in corpus_ids:
        #print(f'\t Document ID: {doc_id}')
        #print(f'\t Text: "{corpus_df.loc[doc_id]["text"]}"')
        # find row index of doc_id
        row_index = corpus_df.index.get_loc(doc_id)
        sorted_scores_task_2.append((doc_id, scores_task_2[index, row_index]))
        #print(f'\t Score: {scores_task_2[index, row_index]}')
        #print()

    sorted_scores_task_2 = sorted(sorted_scores_task_2, key = lambda x: x[1], reverse=True)[:10]

    for entry in sorted_scores_task_2:
        doc_id, score = entry[0], entry[1]
        print(f'\t Document ID: {doc_id}')
        print(f'\t Text: "{corpus_df.loc[doc_id]["text"]}"')
        print(f'\t Score: {score}')
    
    break

QueryID: 1115776:
Query: "what is an aml surveillance analyst"
Top 10 documents:
1462410
	 Document ID: 8106324
	 Text: "Be the first to see new Aml Surveillance Analyst jobs My email: Also get an email with jobs recommended just for me Anti Money Laundering Analyst salaries in United States"
	 Score: 30.13295554433937
	 Document ID: 8106323
	 Text: "NEXT Financial Surveillance Analyst: 1 NEXT Financial Surveillance Analyst salaries $54,259 Philadelphia Stock Exchange Surveillance Analyst: 1 Philadelphia Stock Exchange Surveillance Analyst salaries"
	 Score: 25.13075050876806
	 Document ID: 4314171
	 Text: "Today's top 9449 Aml Analyst jobs in United States. Leverage your professional network, and get hired. New Aml Analyst jobs added daily."
	 Score: 23.928872683499506
	 Document ID: 600966
	 Text: "average casino surveillance analyst salaries for job postings in reno nv are 19 % lower than average casino surveillance analyst salaries for job postings nationwideverage casino surveilla

In [178]:
#Old code that computes the scores using nested for loop
"""scores_task_22 = []

for row in test_data_2.iloc:
    corpus_ids = extract_numbers(row["corpus-id"])
    query_id = row["query-id"]
    query_index = queries_df.index.get_loc(query_id)

    query = queries_df["text"].iloc[query_index]
    query = helpers.tokenize(query,stemmer)
    scores_inter = []
    
    for i, corpus_id in enumerate(corpus_ids):
        corpus_index = corpus_df.index.get_loc(corpus_id)
        score = helpers.bm25(documents[corpus_index], query, idfs, avg_doc_length, k1 = 1, b = 0.75)
        scores_inter.append((corpus_id, score))
    sorted_scores = sorted(scores_inter, key=lambda x: x[1], reverse=True)[:10]
    scores_task_22.append(sorted_scores)    

scores_task_22[0]"""

'scores_task_22 = []\n\nfor row in test_data_2.iloc:\n    corpus_ids = extract_numbers(row["corpus-id"])\n    query_id = row["query-id"]\n    query_index = queries_df.index.get_loc(query_id)\n\n    query = queries_df["text"].iloc[query_index]\n    query = helpers.tokenize(query,stemmer)\n    scores_inter = []\n    \n    for i, corpus_id in enumerate(corpus_ids):\n        corpus_index = corpus_df.index.get_loc(corpus_id)\n        score = helpers.bm25(documents[corpus_index], query, idfs, avg_doc_length, k1 = 1, b = 0.75)\n        scores_inter.append((corpus_id, score))\n    sorted_scores = sorted(scores_inter, key=lambda x: x[1], reverse=True)[:10]\n    scores_task_22.append(sorted_scores)    \n\nscores_task_22[0]'